提前准备好：
- 训练数据（题目+标准答案）
- 云端GPU（RunPod）

In [ ]:
## 安装所需Python数据库：Unsloth（其中包含Torch，TRL等其他库）
pip install unsloth

In [ ]:
## FastLanguageModel：用于加载大语言模型的Class
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/aurore/.local/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
## 加载基础模型（Llama 3.3 70B Instruct）
## load_in_4bit = True：用4bit形式加载（必选，否则GPU直接爆炸）
## dtype：忽略，留在None不动
## max_seq_length：模型最多可以接受多少个词的输入（建议4096或8192，取决于训练数据到底多长）
max_seq_length = 8192
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(model_name="unsloth/Llama-3.3-70B-Instruct",
                                                     max_seq_length=max_seq_length,
                                                     dtype=dtype,
                                                     load_in_4bit=load_in_4bit)

In [ ]:
## 设置LoRA，全部保持不动即可
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha=16,
    lora_dropout=0,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
## 加载数据集

from unsloth import to_sharegpt
from datasets import load_dataset

dataset = load_dataset('csv', data_files='your_dataset_here.csv', split='train').select_columns(['question', 'answer']) # 换成你数据集的列表名称
# 换成你的数据集

In [ ]:
from unsloth import standardize_sharegpt 
dataset = standardize_sharegpt(dataset) # 转换成模型可以理解的格式
print(dataset[0]) # 检查格式

KeyError: 'conversations'

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", 
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True,        # Maps <|im_end|> to <|eot_id|> instead
) # 转换成Llama 3（我们的模型）可以理解的格式

def formatting_prompts_func(examples):
    convos = []
    for question, answer in zip(examples['Question'], examples['Answer']): # 换成你的格式
        tool_user = {
            "content": f"""
    你的System Prompt（系统指令）
""",
            "role": "system"
        }
        ques_user = {
            "content": f"{question}", # 题目
            "role": "user"
        }
        assistant = {
            "content": f"{answer}", # 标准答案
            "role": "assistant"
        }
        convos.append([tool_user, ques_user, assistant])

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Apply the formatting on dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset[0]) # 检验格式转换

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

{'Question': 'Discuss the view that price stability should be the main aim of government.', 'Answer': ' The focus on rising GDP can also help decrease unemployment, maintain a healthy balance of payments and help a country become more competitive internationally. As the potential and capabilities of the economy increase, the economy will operate more efficiently. Greater opportunities will be available for the citizens of the country and it will attract foreign investment. Some inflation may be acceptable in order to pursue the objective of growth. Secondly, it could be argued that economic growth should be the main aim of government. For developing countries and arguably developed economies, economic growth can be seen to be the main aim. Economic growth is measured by the rise of GDP. In fig. 1 you can see the impact of an increase in aggregate demand on real GDP and the price level. Depending on the levels of spare capacity, increases in economic growth may not have a significant im

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 开始训练

trainer = SFTTrainer(
        model = model, # 模型名称（保持不动）
    tokenizer=tokenizer,
    train_dataset = dataset, # 你数据集的名称（保持不动）
    dataset_text_field = "text",
    max_seq_length = 2048, # 模型可接受的最长输入（2048/4096/8192都可以）
    dataset_num_proc = 2,
    packing=False,
    ## 训练开始之后会在屏幕上打出训练步数和一个叫Loss的数据 Loss表示的是模型目前生成的答案有多靠近标准答案 Loss小于0.5的时候可以停止训练
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps=5,
        num_train_epochs=1, # 训练时长 - 1-3都可以 取决于数据集有多大（大一点的数据集可以多训练一会儿）
        learning_rate = 1e-5, # 1 * 10^(-5) 这个是模型的学习速度 建议稍微小一点否则会出现过度拟合（模型只会模仿数据集说话不会学习）
        fp16 = not is_bfloat16_supported(), # 保持不动就可以啦
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        #save_strategy = "epoch",
        #report_to= "wandb"
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/352 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/352 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/352 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/352 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer_stats = trainer.train() # 开始训练

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 352 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 132
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.048700
2,1.975200
3,1.996600
4,1.823700
5,1.922200
6,1.890600
7,1.954300
8,2.068600
9,1.898400
10,1.838900


In [ ]:
model.save_pretrained('al_econ_test') # 把模型保存到叫al_econ_test的文件夹里 随便选个名字都可以 之后下载一下这个文件夹
tokenizer.save_pretrained('al_econ_test')

('al_econ_test/tokenizer_config.json',
 'al_econ_test/special_tokens_map.json',
 'al_econ_test/tokenizer.json')

In [ ]:
## 测试模型（可选）

question = 'Discuss whether expenditure-reducing policies are likely to reduce the current account deficit on the balance of payments for an economy with a floating exchange rate.'

chat = [{
            "content": f"You are an A-Level economics teacher with full knowledge of the A-Level economics syllabus. Your task is to write a sample essay that will obtain top marks according to the A-Level mark scheme on the given essay question.\n",
            "role": "system" # 换成你训练模型用的system prompt
        },
        {
            "content": question,
            "role": "user"
        }]

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer.apply_chat_template(
    chat,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True) # max_new_tokens是模型给出的答案的长度限制 如果模型总是突然挂机的话可以稍微大一点
response = tokenizer.batch_decode(outputs)[0]
print(response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an A-Level economics teacher with full knowledge of the A-Level economics syllabus. Your task is to write a sample essay that will obtain top marks according to the A-Level mark scheme on the given essay question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Discuss whether expenditure-reducing policies are likely to reduce the current account deficit on the balance of payments for an economy with a floating exchange rate.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The current account (CA) is part of the balance of payments (BOP). The CA records four types of transactions: visible, invisible, investment and transfer payments. Visible exports and imports are recorded as trade in goods. Invisible exports and imports include services, interest and profit payments. Investment payments include foreign direct investment into a country and investment payments by citizens of a country to foreign enterprises